In [1]:
import geopandas as gpd
shapefile = 'ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns.
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [2]:

print(gdf[gdf['country'] == 'Antarctica'])


        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  


In [3]:
#We drop the row for Antarctica as it not required!
gdf = gdf.drop(gdf.index[159])

In [4]:
import pandas as pd
datafile = 'share-of-adults-defined-as-obese.csv'
df = pd.read_csv(datafile, names = ['entity', 'code', 'year', 'per_cent_obesity'], skiprows = 1)
df.head()

#This dataset has data on the share of adults who are obese(1975-2016)

,entity,code,year,per_cent_obesity
0,Afghanistan,AFG,1975,0.5
1,Afghanistan,AFG,1976,0.5
2,Afghanistan,AFG,1977,0.6
3,Afghanistan,AFG,1978,0.6
4,Afghanistan,AFG,1979,0.6


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8316 entries, 0 to 8315
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   entity            8316 non-null   object 
 1   code              7980 non-null   object 
 2   year              8316 non-null   int64  
 3   per_cent_obesity  8316 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 260.0+ KB


In [13]:
df[df['code'].isnull()]

,entity,code,year,per_cent_obesity
42,Africa,NaN,1975,2.0
43,Africa,NaN,1976,2.1
44,Africa,NaN,1977,2.2
45,Africa,NaN,1978,2.3
46,Africa,NaN,1979,2.3
...,...,...,...,...
8185,Western Pacific,NaN,2012,5.2
8186,Western Pacific,NaN,2013,5.5
8187,Western Pacific,NaN,2014,5.8
8188,Western Pacific,NaN,2015,6.1


In [24]:
print(df['per_cent_obesity'].max())
print(df['per_cent_obesity'].min())

61.0
0.1


In [12]:
#Finding the name of entities in which code column has NaN values:-
df[df['code'].isnull()].entity.unique()

array(['Africa', 'Americas', 'Eastern Mediterranean', 'Europe', 'Global',
       'South-East Asia', 'Sudan (former)', 'Western Pacific'],
      dtype=object)

In [40]:
#Filter data for year 2016.
df_2016 = df[df['year'] == 2016]
#Perform left merge to preserve every row in gdf.
#Thus, every country in GeoDataframe gdf is preserved in the merged dataframe, even if the corresponding row/s are missing in the pandas dataframe df.
merged = gdf.merge(df_2016, left_on = 'country_code', right_on = 'code', how = 'left')

#Replace NaN values to string 'No data'.
#This is not required now as NaN is a valid JSON object now!
#merged.fillna('No data', inplace = True)

This merged file is a GeoDataframe object that can be rendered using a geopandas module. But, to incorporate data visualisation interactivity, we will use Bokeh. Bokeh consumes GeoJSON format which represents geographical features with JSON. GeoJSON describes points, lines and polygons (called Patches in Bokeh) as a collection of features. We therefore convert the merged file to GeoJSON format.

In [41]:
import json
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [42]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [43]:
from bokeh.resources import INLINE
import bokeh.io

bokeh.io.output_notebook(INLINE)

Loading BokehJS ...

In [44]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

In [45]:
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

In [46]:
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

In [47]:
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40)
#color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = '#d9d9d9')

In [48]:
#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

In [49]:
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

In [50]:
#Create figure object.
p = figure(title = 'Share of adults who are obese, 2016', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

In [51]:
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'per_cent_obesity', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

GlyphRenderer(id='1280', ...)

In [52]:
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

#natural grey indicates missing data

Loading BokehJS ...

In [53]:
#Creating dynamic map that updates data based on year selected in range 1975-2016
#Also adding hover tool to view details by hovering over a specific country/region

from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

In [54]:
#Define function that returns json_data for year selected by user.
    
def json_data(selectedYear):
    yr = selectedYear
    df_yr = df[df['year'] == yr]
    merged = gdf.merge(df_yr, left_on = 'country_code', right_on = 'code', how = 'left')
    #merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

In [55]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data(2016))

In [56]:
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

In [57]:
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

In [58]:
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40)
#color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = '#d9d9d9')

In [59]:
#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

In [60]:
#Add hover tool
hover = HoverTool(tooltips = [ ('Country/region','@country'),('% obesity', '@per_cent_obesity')])

In [61]:
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

In [63]:
#Create figure object.
p = figure(title = 'Share of adults who are obese, 2016', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

In [64]:
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'per_cent_obesity', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

GlyphRenderer(id='1424', ...)

In [65]:
#Specify layout
p.add_layout(color_bar, 'below')

In [66]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    p.title.text = 'Share of adults who are obese, %d' %yr

In [67]:
# Make a slider object: slider 
slider = Slider(title = 'Year',start = 1975, end = 2016, step = 1, value = 2016)
slider.on_change('value', update_plot)

In [68]:
# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

In [70]:
#Display plot inline in Jupyter notebook
output_notebook()
#Display plot
show(layout)

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



The plot does not update when you change slider value in your Jupyter Notebook. To view this application in interactive mode you need to set up a local Bokeh server. Open a command line window in your current directory and execute 

bokeh serve --show filename.ipynb 

Click on the link prompted to open the application in your browser.